<a href="https://colab.research.google.com/github/aryagagas/machine-learning-and-deep-learning-assignment/blob/main/04_01_Air_Arya_Gagasan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

Preparation biasa dilakukan untuk mempersiapkan data sebelum masuk dalam tahap pemodelan. <br>
Berikut adalah tahapan yang akan dilalui pada data `SC_HW1_bank_data.csv` sebelum tahap pemodelan :
1. Import Library
2. Input Dataset
3. Preprocessing
4. Train-Test Split

## Import Library

In [ ]:
import pandas as pd
import numpy as np

## Input Dataset

In [ ]:
#Membaca data dan memasukkannya ke dalam bentuk Pandas Dataframe
df = pd.read_csv('https://raw.githubusercontent.com/Rietaros/kampus_merdeka/main/SC_HW1_bank_data.csv')

In [ ]:
#Jalankan code untuk mengecek nama kolom yang tersedia
df.columns

Index(['RowNumber', 'CustomerId', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
#Hilangkan kolom yang dirasa tidak relevan dengan model (contoh: ID). None dapat diisi dengan nama-nama kolom yang akan digunakan.
#Contoh df = df[['X1','X2', 'Y']].copy()

#START CODE
df = df[['Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']].copy()

## Preprocessing

In [ ]:
#Lakukan One-Hot Encoder untuk data categorical, dengan fungsi pandas get_dummies

#START CODE
df = pd.get_dummies(df, columns=['Geography', 'Gender'], prefix=['Geo', 'Gen'])

In [ ]:
#Pisahkan mana X (feature) dengan Y,
#Y adalah kolom "Exited"

#START CODE
X = df.drop('Exited', axis=1)
y = df.loc[:, 'Exited']

In [ ]:
#Lakukan Scaler dan/atau Noermalisasi jika diperlukan
from sklearn.preprocessing import MinMaxScaler

#START CODE
scaler = MinMaxScaler()
X_transform = scaler.fit_transform(X)

In [ ]:
#Ini digunakan jika dilakukan scaler/Normalisas. Jika tidak, code ini bisa dilewat dan diganti dengan code yang ada di dalam komen
X_transform = pd.DataFrame(X_transform, columns = X.columns)
#X_transform = X.copy()

## Train-Test Split

In [ ]:
#Split menjadi train dan test dengan test_size 25%
#Tidak perlu mengubah code ini

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_transform,y,test_size = 0.25,random_state = 123)

# Modeling

## Model1

### Soal :
Jelaskan secara Singkat Model pertama yang digunakan!

> KNN bekerja dengan cara mencari sejumlah K tetangga terdekat dari suatu data uji (data yang akan diprediksi) dalam ruang fitur berdasarkan jarak euclidean atau metrik lainnya. Kemudian, berdasarkan mayoritas kelas dari tetangga-tetangga tersebut, data uji akan diklasifikasikan ke dalam kelas yang paling mendominasi di antara tetangga-tetangga tersebut.



In [ ]:
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

# model1 = LogisticRegression()
# params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}


#START CODE
from sklearn.neighbors import KNeighborsClassifier

model1 = KNeighborsClassifier()

# Definisi hyperparameter yang akan dituning
params = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model1,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}

In [ ]:
#lakukan evaluasi dengan beberapa maetric di bawah ini
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1983
           1       0.67      0.33      0.45       517

    accuracy                           0.83      2500
   macro avg       0.76      0.65      0.67      2500
weighted avg       0.81      0.83      0.80      2500


[[1897   86]
 [ 344  173]]

0.828


## Model2
### Soal :
Jelaskan secara Singkat Model ke-2 yang digunakan!

> Random Forest bekerja dengan cara membangun sejumlah pohon keputusan acak (forest) berdasarkan subset data pelatihan yang dipilih secara acak dan kemudian menggabungkan hasil prediksi dari masing-masing pohon untuk menghasilkan hasil akhir. Hal ini membantu mengurangi overfitting yang mungkin terjadi dalam model pohon keputusan tunggal.

In [ ]:
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

#from sklearn.linear_model import LogisticRegression
#model1 = LogisticRegression()
#params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}


#START CODE
from sklearn.ensemble import RandomForestClassifier

# Tentukan model
model2 = RandomForestClassifier()

# Tentukan daftar hyperparameter yang akan diuji
params = {
    'n_estimators': [100, 200, 300],  # Jumlah pohon keputusan dalam ensemble
    'max_depth': [None, 10, 20],  # Kedalaman maksimum setiap pohon
    'min_samples_split': [2, 5, 10],  # Jumlah sampel minimum yang diperlukan untuk membagi node
    'min_samples_leaf': [1, 2, 4]  # Jumlah sampel minimum yang diperlukan untuk menjadi leaf node
}
#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model2,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 100}

In [ ]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1983
           1       0.79      0.44      0.57       517

    accuracy                           0.86      2500
   macro avg       0.83      0.71      0.74      2500
weighted avg       0.85      0.86      0.84      2500


[[1923   60]
 [ 288  229]]

0.8608


## Model3
### Soal :
Jelaskan secara Singkat Model ke-3 yang digunakan!

> Pada dasarnya, Gradient Boosting bekerja dengan cara menggabungkan sejumlah model lemah (dalam hal ini, pohon keputusan) menjadi model yang kuat. Setiap pohon yang ditambahkan ke ensemble fokus pada kesalahan prediksi yang dibuat oleh pohon-pohon sebelumnya. Dengan cara ini, model secara bertahap "mempelajari" data dengan mengurangi kesalahan prediksi.

In [ ]:
#Pilih salah satu metode Machine Leaarning
#Model Machine Learning dapat dipanggil terlebih dahulu melalui library yang digunakan. Gunakan library scikit learn seperti pada contoh

#Contoh pemanggilan library dan penggunaannya dalam model

#from sklearn.linear_model import LogisticRegression
#model1 = LogisticRegression()
#params = { "tol": [0.1,0.01,0.001], 'C':[0.5,1.0,1.5,2.0]}


#START CODE
# Import library yang dibutuhkan
from sklearn.ensemble import GradientBoostingClassifier

# Tentukan model Gradient Boosting
model3 = GradientBoostingClassifier()

# Tentukan daftar hyperparameter yang akan diuji
params = {
    'n_estimators': [50, 100, 200],  # Jumlah estimator (pohon) dalam ensemble
    'learning_rate': [0.01, 0.1, 0.2],  # Tingkat pembelajaran
    'max_depth': [3, 4, 5]  # Kedalaman maksimum pohon
}
#END CODE

#Lakukan parameter tuning sesuai hyperparameter yang dibutuhkan
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(
             estimator= model3,
             param_grid= params,
             scoring = 'accuracy',
             n_jobs = 10, # core cpu yang digunakan
             cv = 10 # 3-fold cross validation (artinya kita melakukan iterasi model sebanyak 3 kali)
            )

grid.fit(X_train,y_train)
grid.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}

In [ ]:
#lakukan evaluasi
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = grid.predict(X_test)

print(classification_report(y_test,y_pred))
print("")
print(confusion_matrix(y_test,y_pred))
print("")
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1983
           1       0.78      0.51      0.62       517

    accuracy                           0.87      2500
   macro avg       0.83      0.74      0.77      2500
weighted avg       0.86      0.87      0.86      2500


[[1906   77]
 [ 251  266]]

0.8688


## Tarik Kesimpulan Model Mana yang terbaik beserta alasannya

Berdasarkan hasil evaluasi, Model 3 (Gradient Boosting Classifier) adalah model terbaik untuk tugas ini. Ini dapat dijelaskan sebagai berikut:

**Model 1 (K-Nearest Neighbors):**

> Parameter Terbaik:

```
{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}
```

```
              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1983
           1       0.67      0.33      0.45       517

    accuracy                           0.83      2500
   macro avg       0.76      0.65      0.67      2500
weighted avg       0.81      0.83      0.80      2500


[[1897   86]
 [ 344  173]]

0.828
```

**Model 2 (Random Forest Classifier):**

> Parameter Terbaik:

```
{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
```

```
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1983
           1       0.79      0.44      0.57       517

    accuracy                           0.86      2500
   macro avg       0.83      0.71      0.74      2500
weighted avg       0.85      0.86      0.84      2500


[[1923   60]
 [ 288  229]]

0.8608
```

**Model 3 (Gradient Boosting Classifier):**

> Parameter Terbaik:

```
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
```

```
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1983
           1       0.78      0.51      0.62       517

    accuracy                           0.87      2500
   macro avg       0.83      0.74      0.77      2500
weighted avg       0.86      0.87      0.86      2500


[[1906   77]
 [ 251  266]]

0.8688
```

**Kesimpulan:** <br>
> Model 3 **(Gradient Boosting Classifier)** adalah yang terbaik karena memiliki akurasi tertinggi (0.8688) atau ketika dibulatkan menjadi **87%** dan F1-score tertinggi untuk Kelas 1 (0.62). Meskipun Model 2 **(Random Forest Classifier)** memiliki akurasi yang baik, Model 3 memiliki presisi dan recall yang lebih baik untuk kelas 1, sehingga lebih cocok untuk tugas ini. Model 1 **(K-Nearest Neighbors)** memiliki kinerja yang lebih rendah dibandingkan dengan Model 2 dan Model 3.

> Model 3 adalah pilihan terbaik karena mampu menghasilkan prediksi yang baik dengan keseimbangan antara presisi dan recall, yang penting dalam pemodelan klasifikasi. Model ini juga memiliki parameter terbaik yang telah di-tune untuk mencapai kinerja maksimal.